# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `各ノードを Root / Inner / Leaf に分類して返す。`
* 入力: `Tree(id int primary key, p_id int)`
* 出力: `id, type`

  * `p_id IS NULL` → `"Root"`
  * 親あり かつ 子なし → `"Leaf"`
  * 親あり かつ 子あり → `"Inner"`

## 2) 最適解（単一クエリ）

> 子を持つ ID の集合だけを `DISTINCT` で事前に作り、元表へ左結合して `CASE` 判定。
> （この問題はウィンドウ不要。`DISTINCT` は `GROUP BY + COUNT(*)` より軽くなるケースが多い）

```sql
WITH has_child AS (
  SELECT DISTINCT p_id AS id
  FROM Tree
  WHERE p_id IS NOT NULL
)
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN hc.id IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN has_child AS hc
  ON hc.id = t.id;

Runtime 204 ms
Beats 72.79%

```

### 代替（相関 EXISTS）

> セミジョイン最適化と早期終了が期待できる形。`Tree(p_id)` に索引があれば速いです。

```sql
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN EXISTS (SELECT 1 FROM Tree ch WHERE ch.p_id = t.id) THEN 'Inner'
    ELSE 'Leaf'
  END AS type
FROM Tree AS t;

Runtime 198 ms
Beats 87.27%

```

## 3) 要点解説

* **方針**: 「子の有無」を一度だけ判定してから結合 → `CASE` で型を決める。
* **NULL の扱い**: ルート判定は必ず `p_id IS NULL` で明示。
* **CTE の最適化**: PG16 では本クエリの CTE は基本的に **インライン化** され、派生表と同等に最適化されます。
* **索引**（最重要）:

  ```sql
  -- 仕様上 id は一意想定
  ALTER TABLE Tree
    ADD PRIMARY KEY (id);
  CREATE INDEX IF NOT EXISTS idx_tree_pid ON Tree (p_id);
  ```

  `LEFT JOIN has_child` も `EXISTS` も `p_id` インデックスで大きく効きます。
* **スケール時のヒント**: 極端に「星型（root の直下に大量の子）」など分布が偏る場合は `EXISTS` 版が有利なことがあります。必ず実測で比較を。

## 4) 計算量（概算）

* `SELECT DISTINCT p_id`：**O(N)**～**O(N log N)**
* `LEFT JOIN`：インデックスありで **O(N)** 近似
* 全体：**O(N)**～**O(N log N)**（`EXISTS` 版はヒット即終了のため平均的に有利なことあり）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Tree id p_id]
  B[has_child DISTINCT p_id]
  C[LEFT JOIN で結合]
  D[CASE で Root Inner Leaf]
  E[出力 id type]
  A --> B
  A --> C
  B --> C
  C --> D
  D --> E
```

さらに数％〜数十％詰められる可能性があるポイントを、**実装（SQL）** と **実運用（索引・設定）** に分けて挙げます。小さく試せる順にどうぞ。

---

## すぐ試せる SQL バリエーション

### 1) `LATERAL + LIMIT 1`（存在チェックを“最初の1件”で強制停止）

`EXISTS` と同趣旨ですが、**「最初の一致で打ち切る」** をクエリ形で明示できます。`Tree(p_id)` に索引があると特に効きやすいです。

```sql
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN ch.exists IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN LATERAL (
  SELECT 1 AS exists
  FROM Tree AS c
  WHERE c.p_id = t.id
  LIMIT 1
) ch ON TRUE;

Runtime 201 ms
Beats 80.29%

```

**ねらい**

* 相関副問い合わせと同じ“セミジョイン”挙動を誘発しつつ、`LIMIT 1` で**確実に早期終了**。
* プランナーは `Index Scan`（`p_id`）を選びやすく、**ランダム I/O 最小化**に寄与。

---

### 2) `UNION ALL` 三分割（選択性に偏りがあるとき）

スキャン回数は増えますが、条件が強い枝では一気に削れます。ルートが少数、葉が多数など**分布が偏っている**場合に有利なことが多いです。

```sql
SELECT id, 'Root' AS type
FROM Tree
WHERE p_id IS NULL

UNION ALL
SELECT t.id, 'Inner'
FROM Tree AS t
WHERE t.p_id IS NOT NULL
  AND EXISTS (SELECT 1 FROM Tree AS ch WHERE ch.p_id = t.id)

UNION ALL
SELECT t.id, 'Leaf'
FROM Tree AS t
WHERE t.p_id IS NOT NULL
  AND NOT EXISTS (SELECT 1 FROM Tree AS ch WHERE ch.p_id = t.id);

Runtime 250 ms
Beats 18.17%

```

---

### 3) `DISTINCT` 版を明示的に **NOT MATERIALIZED**（CTE最適化の安全札）

PG16 では原則インライン化されますが、念のため明示して**余計なマテリアライズ回避**を保証できます。

```sql
WITH NOT MATERIALIZED has_child AS (
  SELECT DISTINCT p_id AS id
  FROM Tree
  WHERE p_id IS NOT NULL
)
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN hc.id IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN has_child AS hc
  ON hc.id = t.id;

syntax error at or near "NOT"
LINE 1: WITH NOT MATERIALIZED has_child AS (
             ^
```

---

## 索引と統計（最重要）

```sql
-- 仕様: id は一意
ALTER TABLE Tree
  ADD PRIMARY KEY (id);

-- 子探索用。NULL は不要なので部分索引が有効（ヒープ・インデックス小型化）
CREATE INDEX IF NOT EXISTS idx_tree_pid_nn ON Tree (p_id) WHERE p_id IS NOT NULL;
```

**メモ**

* `EXISTS` / `LATERAL` / `DISTINCT` いずれの形でも **`p_id` 索引が効く**のが最大の勝ち筋。
* データ分布が偏る場合は、`p_id` の統計をリッチに：

  ```sql
  ALTER TABLE Tree ALTER COLUMN p_id SET STATISTICS 1000;
  ANALYZE Tree;
  ```

  選択性推定が改善し、**適切な Index Scan / Join 戦略**を選びやすくなります。

---

## 実行設定の小技（計測環境次第で効く）

* **JIT**: 小〜中規模クエリは JIT の起動コストが負けることが多いです。計測時はオフで比較。

  ```sql
  SET jit = off;
  ```
* **work_mem**: `DISTINCT`（ハッシュ重複排除）やハッシュ結合がディスクに溢れるなら増量。

  ```sql
  SET work_mem = '64MB';  -- テストセッションだけ一時的に
  ```
* **VACUUM / ANALYZE**: 断片化・古い統計があると悪化。実データでの比較前に一度実施。

---

## どれを採用すべき？

1. まずは **`EXISTS` 版（198ms）を基準**に、
2. **`LATERAL + LIMIT 1`** を試して比較（同等〜数%改善するケースが多い）、
3. 分布に偏りがあるなら **`UNION ALL` 三分割** も候補、
4. いずれも **`idx_tree_pid_nn`（部分索引）+ ANALYZE** は必須。

> 体感では、`p_id` 側にしっかり索引・統計が効いていれば、
> **`EXISTS` ≒ `LATERAL LIMIT 1` > `DISTINCT JOIN`** の順で良いことが多いです。
> 逆にメモリが潤沢で一括処理寄りなら `DISTINCT JOIN` が安定、という傾向です。

必要なら `EXPLAIN (ANALYZE, BUFFERS)` の結果を貼っていただければ、プランごとに**停止点（早期終了）・ヒープヒット率・メモリ落ち**まで具体的に詰めます。

原因は **キーワードの位置** です。
PostgreSQL の構文は

```
WITH cte_name AS [NOT] MATERIALIZED ( ... )
```

であり、`WITH NOT MATERIALIZED cte_name AS` ではありません。

## 正しい書き方

```sql
WITH has_child AS NOT MATERIALIZED (
  SELECT DISTINCT p_id AS id
  FROM Tree
  WHERE p_id IS NOT NULL
)
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN hc.id IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN has_child AS hc
  ON hc.id = t.id;

Runtime 210 ms
Beats 57.91%

```

## 補足

* `AS NOT MATERIALIZED` は **PG 12+** で有効（16.6 ならOK）。
* 目的は「CTE を強制的にインライン化」して不要なマテリアライズを避けることです。
* 同じ効果を狙うなら、**派生表**でもOKです（こちらは構文がシンプル）:

```sql
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN hc.id IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN (
  SELECT DISTINCT p_id AS id
  FROM Tree
  WHERE p_id IS NOT NULL
) AS hc
  ON hc.id = t.id;

Runtime 224 ms
Beats 35.93%

```

これでエラーは解消します。

